Судебная практика https://sudact.ru/

В составе раздела – обзор решений судов по наиболее популярным правовым вопросам и ситуациям.

По административным делам - https://sudact.ru/practice/sudebnaya-praktika-po-administrativnym-delam/

По гражданским делам - https://sudact.ru/practice/sudebnaya-praktika-po-grazhdanskim-delam/

По уголовным делам - https://sudact.ru/practice/sudebnaya-praktika-po-ugolovnym-delam/

In [1]:
import numpy as np

In [2]:
sections = np.array([
    ('По административным делам', 'https://sudact.ru/practice/sudebnaya-praktika-po-administrativnym-delam/'),
    ('По гражданским делам', 'https://sudact.ru/practice/sudebnaya-praktika-po-grazhdanskim-delam/'),
    ('По уголовным делам', 'https://sudact.ru/practice/sudebnaya-praktika-po-ugolovnym-delam/')
])

In [3]:
sections[:, 1]

array(['https://sudact.ru/practice/sudebnaya-praktika-po-administrativnym-delam/',
       'https://sudact.ru/practice/sudebnaya-praktika-po-grazhdanskim-delam/',
       'https://sudact.ru/practice/sudebnaya-praktika-po-ugolovnym-delam/'],
      dtype='<U72')

1. Заходим на страницу раздела
2. Получаем список подразделов по применению конкретных норм
3. Получаем список решений 
4. Переходим на следующую страницу
5. Повторяем пункты 3, 4, пока есть страницы

Для конкретного решения у нас есть структурированная информация:
* название (содержит номер и дату)
* ссылка на страницу
* раздел
* подраздел
* суд
* html страницы

In [ ]:
#!pip install scrapy

In [ ]:
#!pip install pyopenssl==22.0.0

In [ ]:
#!pip install crochet

In [4]:
import sqlite3

In [5]:
con = sqlite3.connect("sudactru.db")

cur = con.cursor()

sql = '''
CREATE TABLE IF NOT EXISTS sections(
    id INTEGER PRIMARY KEY,
    name TEXT,
    href TEXT
)
'''

cur.executescript(sql)
con.commit()

In [148]:
#import pandas as pd

In [149]:
sql = '''
INSERT INTO sections(name, href) 
    VALUES (?, ?)
'''
cur.executemany(sql, sections)
con.commit()

In [150]:
res = cur.execute("SELECT * FROM sections;")
res.fetchall()

[(1,
  'По административным делам',
  'https://sudact.ru/practice/sudebnaya-praktika-po-administrativnym-delam/'),
 (2,
  'По гражданским делам',
  'https://sudact.ru/practice/sudebnaya-praktika-po-grazhdanskim-delam/'),
 (3,
  'По уголовным делам',
  'https://sudact.ru/practice/sudebnaya-praktika-po-ugolovnym-delam/')]

In [151]:
class SqliteSectionPipeline:

    def __init__(self):
        self.con = sqlite3.connect('sudactru.db')
        self.cur = self.con.cursor()
        self.cur.execute("""
        CREATE TABLE IF NOT EXISTS subsections(
            id INTEGER PRIMARY KEY,
            name TEXT,
            href TEXT UNIQUE,
            section_id INT,
            FOREIGN KEY (section_id) REFERENCES sections(id)
        )
        """)


    def process_item(self, item, spider):
        
        self.cur.execute("""
            INSERT INTO subsections (name, href, section_id) VALUES (?, ?, ?)
        """,
        (
            item['name'],
            item['href'],
            item['section_id']
        ))

        self.con.commit()

        return item


In [9]:
# scrape webpage
import scrapy
from scrapy.crawler import CrawlerRunner
# text cleaning
import re
# Reactor restart
from crochet import setup, wait_for
setup()

In [153]:
class SudactSectionSpider(scrapy.Spider):
    """
    Парсинг списка страниц на применения разных норм в рамках данного раздела.
    """
    def __init__(self):
        self.con = sqlite3.connect('sudactru.db')
        self.cur = self.con.cursor()
        
    name = "SudactSectionSpider"
    allowed_domains = ['sudact.ru']
    start_urls = list(sections[:, 1])
    
    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.SqliteSectionPipeline': 1
        },
        "DOWNLOAD_DELAY": 0.25,
        "LOG_LEVEL": "WARNING",
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
    }

    def parse(self, response):
        current_url = response.request.url   
        h1 = response.xpath('//h1/text()').get()    
        print(h1)
        res = self.cur.execute("SELECT id FROM sections WHERE href = ?", (current_url,))
        section_id = res.fetchone()[0]
            
        for link in response.css('.wos'):
            yield {'name': link.css('::text').get(), 
                   'href': link.css('a').attrib["href"],
                   'section_id': section_id
                  }

@wait_for(10)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(SudactSectionSpider)
    return d

In [154]:
run_spider()

Судебная практика по административным делам
Судебная практика по гражданским делам
Судебная практика по уголовным делам


In [234]:
res = cur.execute("SELECT href FROM subsections;")
list(map(lambda x: 'https://sudact.ru' + x[0], res.fetchall()))

['https://sudact.ru/practice/amnistiya/',
 'https://sudact.ru/practice/dokazatelstva/',
 'https://sudact.ru/practice/halatnost/',
 'https://sudact.ru/practice/inostrannye-grazhdane/',
 'https://sudact.ru/practice/kleveta/',
 'https://sudact.ru/practice/kommercheskij-podkup/',
 'https://sudact.ru/practice/kontrabanda/',
 'https://sudact.ru/practice/mery-presecheniya/',
 'https://sudact.ru/practice/narushenie-prav-invalidov/',
 'https://sudact.ru/practice/narushenie-pravil-dorozhnogo-dvizheniya/',
 'https://sudact.ru/practice/nezakonnoe-poluchenie-kredita/',
 'https://sudact.ru/practice/nezakonnoe-predprinimatelstvo/',
 'https://sudact.ru/practice/oskorblenie/',
 'https://sudact.ru/practice/osushestvlenie-predprinimatelskoj-deyatelnosti-bez/',
 'https://sudact.ru/practice/personalnye-dannye/',
 'https://sudact.ru/practice/po-delam-o-huliganstve/',
 'https://sudact.ru/practice/po-delam-ob-iznasilovanii/',
 'https://sudact.ru/practice/po-delam-ob-ubijstve/',
 'https://sudact.ru/practice/po

In [233]:
res = cur.execute("SELECT section_id, COUNT(*) FROM subsections GROUP BY section_id;")
res.fetchall()

[(1, 13), (2, 5), (3, 34)]

In [241]:
class SqliteSubSectionPipeline:
    
    def __init__(self):
        self.con = sqlite3.connect('sudactru.db')
        self.cur = self.con.cursor()
        self.cur.execute("""
        CREATE TABLE IF NOT EXISTS decisions(
            id INTEGER PRIMARY KEY,
            name TEXT,
            href TEXT UNIQUE,
            subsection_id INT,
            justice TEXT,
            short_descr TEXT,
            FOREIGN KEY (subsection_id) REFERENCES subsection(id)
        )
        """)
        # justice - название суда и типа дела, 
        # например "Шатурский городской суд (Московская область) - Гражданское"
        # short_descr - короткое "описание" решения, содержащие название нормы, 
        # в рамках которой рассматривалось данное дело


    def process_item(self, item, spider):
                
        self.cur.execute("""
            INSERT INTO decisions (
                name, 
                href, 
                subsection_id,
                justice,
                short_descr
            ) VALUES (?, ?, ?, ?, ?)
        """,
        (
            item['name'],
            item['href'],
            item['subsection_id'],
            item['justice'],
            item['short_descr']
        ))

        self.con.commit()
        
        return item

In [245]:
class SudactSubSectionSpider(scrapy.Spider):
    """
    Парсинг списка решений по данной правовой норме.
    """
    def __init__(self):
        self.con = sqlite3.connect('sudactru.db')
        self.cur = self.con.cursor()
        
    name = "SudactSubSectionSpider"
    allowed_domains = ['sudact.ru']
    
    res = cur.execute("SELECT href FROM subsections;")
    start_urls = list(map(lambda x: 'https://sudact.ru' + x[0], res.fetchall()))
    #start_urls = ['https://sudact.ru/practice/po-zashite-prav-potrebitelej/?page=49']
        
    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.SqliteSubSectionPipeline': 1
        },
        "DOWNLOAD_DELAY": 0.25,
        "LOG_LEVEL": "WARNING",
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
    }

    def parse(self, response):
        current_url = response.request.url   
        h1 = response.xpath('//h1/text()').get()
        
        res = self.cur.execute("SELECT id FROM subsections WHERE href = ?", (
            current_url.replace('https://sudact.ru', '').split('?')[0],
        ))
        subsection_id = res.fetchone()[0]
           
        print(h1, current_url, subsection_id)
        
        for li in response.css('.results li'):
            yield {'name': li.css('a').css('::text').get(), 
                   'href': li.css('a').attrib["href"],
                   'subsection_id': subsection_id,
                   'justice': li.css('.b-justice::text').get(),
                   'short_descr': ''.join(li.css('::text').getall()[3:])
                  }
        
        next_page = response.css('.page-next a')
        if len(next_page) > 0:
            yield scrapy.Request(
                response.urljoin(current_url.split('?')[0] + next_page.attrib["href"]),
                callback=self.parse
            )

@wait_for(None)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(SudactSubSectionSpider)
    return d

In [246]:
run_spider()

Амнистия - судебная практика https://sudact.ru/practice/amnistiya/ 32
Доказательства - судебная практика https://sudact.ru/practice/dokazatelstva/ 51
Халатность - судебная практика https://sudact.ru/practice/halatnost/ 35
Иностранные граждане - судебная практика https://sudact.ru/practice/inostrannye-grazhdane/ 8
Клевета - судебная практика https://sudact.ru/practice/kleveta/ 19
Коммерческий подкуп - судебная практика https://sudact.ru/practice/kommercheskij-podkup/ 29
Контрабанда - судебная практика https://sudact.ru/practice/kontrabanda/ 40
Меры пресечения - судебная практика https://sudact.ru/practice/mery-presecheniya/ 52
Нарушение прав инвалидов - судебная практика https://sudact.ru/practice/narushenie-prav-invalidov/ 3
Нарушение правил дорожного движения - судебная практика https://sudact.ru/practice/narushenie-pravil-dorozhnogo-dvizheniya/ 49
Незаконное получение кредита - судебная практика https://sudact.ru/practice/nezakonnoe-poluchenie-kredita/ 4
Незаконное предпринимательств

Персональные данные - судебная практика https://sudact.ru/practice/personalnye-dannye/?page=5 7
Судебная практика по делам о хулиганстве https://sudact.ru/practice/po-delam-o-huliganstve/?page=5 31
Судебная практика по делам об изнасиловании https://sudact.ru/practice/po-delam-ob-iznasilovanii/?page=5 26
Амнистия - судебная практика https://sudact.ru/practice/amnistiya/?page=6 32
Доказательства - судебная практика https://sudact.ru/practice/dokazatelstva/?page=6 51
Халатность - судебная практика https://sudact.ru/practice/halatnost/?page=6 35
Иностранные граждане - судебная практика https://sudact.ru/practice/inostrannye-grazhdane/?page=6 8
Клевета - судебная практика https://sudact.ru/practice/kleveta/?page=6 19
Коммерческий подкуп - судебная практика https://sudact.ru/practice/kommercheskij-podkup/?page=6 29
Судебная практика по делам об убийстве https://sudact.ru/practice/po-delam-ob-ubijstve/?page=2 21
Меры пресечения - судебная практика https://sudact.ru/practice/mery-presecheniya

Контрабанда - судебная практика https://sudact.ru/practice/kontrabanda/?page=9 40
Незаконное предпринимательство - судебная практика https://sudact.ru/practice/nezakonnoe-predprinimatelstvo/?page=10 41
Оскорбление - судебная практика https://sudact.ru/practice/oskorblenie/?page=10 2
Осуществление предпринимательской деятельности без регистрации или без разрешения - судебная практика https://sudact.ru/practice/osushestvlenie-predprinimatelskoj-deyatelnosti-bez/?page=10 5
Доказательства - судебная практика https://sudact.ru/practice/dokazatelstva/?page=10 51
Судебная практика по делам о хулиганстве https://sudact.ru/practice/po-delam-o-huliganstve/?page=10 31
Судебная практика по делам об изнасиловании https://sudact.ru/practice/po-delam-ob-iznasilovanii/?page=10 26
Амнистия - судебная практика https://sudact.ru/practice/amnistiya/?page=11 32
Персональные данные - судебная практика https://sudact.ru/practice/personalnye-dannye/?page=10 7
Халатность - судебная практика https://sudact.ru/p

Клевета - судебная практика https://sudact.ru/practice/kleveta/?page=15 19
Персональные данные - судебная практика https://sudact.ru/practice/personalnye-dannye/?page=14 7
Судебная практика по делам об убийстве https://sudact.ru/practice/po-delam-ob-ubijstve/?page=11 21
Меры пресечения - судебная практика https://sudact.ru/practice/mery-presecheniya/?page=15 52
Коммерческий подкуп - судебная практика https://sudact.ru/practice/kommercheskij-podkup/?page=15 29
Нарушение прав инвалидов - судебная практика https://sudact.ru/practice/narushenie-prav-invalidov/?page=13 3
Контрабанда - судебная практика https://sudact.ru/practice/kontrabanda/?page=14 40
Незаконное предпринимательство - судебная практика https://sudact.ru/practice/nezakonnoe-predprinimatelstvo/?page=15 41
Оскорбление - судебная практика https://sudact.ru/practice/oskorblenie/?page=15 2
Осуществление предпринимательской деятельности без регистрации или без разрешения - судебная практика https://sudact.ru/practice/osushestvleni

Судебная практика по делам об изнасиловании https://sudact.ru/practice/po-delam-ob-iznasilovanii/?page=19 26
Контрабанда - судебная практика https://sudact.ru/practice/kontrabanda/?page=18 40
Нарушение правил дорожного движения - судебная практика https://sudact.ru/practice/narushenie-pravil-dorozhnogo-dvizheniya/?page=19 49
Халатность - судебная практика https://sudact.ru/practice/halatnost/?page=20 35
Иностранные граждане - судебная практика https://sudact.ru/practice/inostrannye-grazhdane/?page=20 8
Клевета - судебная практика https://sudact.ru/practice/kleveta/?page=20 19
Доказательства - судебная практика https://sudact.ru/practice/dokazatelstva/?page=19 51
Судебная практика по делам об убийстве https://sudact.ru/practice/po-delam-ob-ubijstve/?page=16 21
Меры пресечения - судебная практика https://sudact.ru/practice/mery-presecheniya/?page=20 52
Коммерческий подкуп - судебная практика https://sudact.ru/practice/kommercheskij-podkup/?page=20 29
Нарушение прав инвалидов - судебная п

...

Самоуправство - судебная практика https://sudact.ru/practice/samoupravstvo/?page=18 20
Приговор, неисполнение приговора - судебная практика https://sudact.ru/practice/prigovor-neispolnenie-prigovora/?page=20 50
Соучастие, предварительный сговор - судебная практика https://sudact.ru/practice/souchastie-predvaritelnyj-sgovor/?page=18 45
Умышленное причинение тяжкого вреда здоровью - судебная практика https://sudact.ru/practice/umyshlennoe-prichinenie-tyazhkogo-vreda-zdorovyu/?page=16 47
Уголовная ответственность несовершеннолетних - судебная практика https://sudact.ru/practice/ugolovnaya-otvetstvennost-nesovershennoletnih/?page=16 30
Разбой - судебная практика https://sudact.ru/practice/razboj/?page=19 28
Злоупотребление должностными полномочиями – судебная практика https://sudact.ru/practice/zloupotreblenie-dolzhnostnymi-polnomochiyami/?page=16 22
Присвоение и растрата - судебная практика https://sudact.ru/practice/prisvoenie-i-rastrata/?page=20 25
Преступное сообщество - судебная практ

Злоупотребление должностными полномочиями – судебная практика https://sudact.ru/practice/zloupotreblenie-dolzhnostnymi-polnomochiyami/?page=22 22
Превышение должностных полномочий - судебная практика https://sudact.ru/practice/prevyshenie-dolzhnostnyh-polnomochij/?page=27 37
Присвоение и растрата - судебная практика https://sudact.ru/practice/prisvoenie-i-rastrata/?page=26 25
Уголовная ответственность несовершеннолетних - судебная практика https://sudact.ru/practice/ugolovnaya-otvetstvennost-nesovershennoletnih/?page=22 30
Преступное сообщество - судебная практика https://sudact.ru/practice/prestupnoe-soobshestvo/?page=43 46
Самоуправство - судебная практика https://sudact.ru/practice/samoupravstvo/?page=25 20
Приговор, неисполнение приговора - судебная практика https://sudact.ru/practice/prigovor-neispolnenie-prigovora/?page=27 50
Соучастие, предварительный сговор - судебная практика https://sudact.ru/practice/souchastie-predvaritelnyj-sgovor/?page=25 45
Умышленное причинение тяжкого 

Разбой - судебная практика https://sudact.ru/practice/razboj/?page=32 28
Злоупотребление должностными полномочиями – судебная практика https://sudact.ru/practice/zloupotreblenie-dolzhnostnymi-polnomochiyami/?page=29 22
Превышение должностных полномочий - судебная практика https://sudact.ru/practice/prevyshenie-dolzhnostnyh-polnomochij/?page=34 37
Присвоение и растрата - судебная практика https://sudact.ru/practice/prisvoenie-i-rastrata/?page=33 25
Самоуправство - судебная практика https://sudact.ru/practice/samoupravstvo/?page=32 20
Уголовная ответственность несовершеннолетних - судебная практика https://sudact.ru/practice/ugolovnaya-otvetstvennost-nesovershennoletnih/?page=29 30
Преступное сообщество - судебная практика https://sudact.ru/practice/prestupnoe-soobshestvo/?page=50 46
Приговор, неисполнение приговора - судебная практика https://sudact.ru/practice/prigovor-neispolnenie-prigovora/?page=34 50
Соучастие, предварительный сговор - судебная практика https://sudact.ru/practice/so

Самоуправство - судебная практика https://sudact.ru/practice/samoupravstvo/?page=39 20
Приговор, неисполнение приговора - судебная практика https://sudact.ru/practice/prigovor-neispolnenie-prigovora/?page=41 50
Соучастие, предварительный сговор - судебная практика https://sudact.ru/practice/souchastie-predvaritelnyj-sgovor/?page=39 45
Уголовная ответственность несовершеннолетних - судебная практика https://sudact.ru/practice/ugolovnaya-otvetstvennost-nesovershennoletnih/?page=36 30
Умышленное причинение тяжкого вреда здоровью - судебная практика https://sudact.ru/practice/umyshlennoe-prichinenie-tyazhkogo-vreda-zdorovyu/?page=37 47
Разбой - судебная практика https://sudact.ru/practice/razboj/?page=40 28
Злоупотребление должностными полномочиями – судебная практика https://sudact.ru/practice/zloupotreblenie-dolzhnostnymi-polnomochiyami/?page=37 22
Превышение должностных полномочий - судебная практика https://sudact.ru/practice/prevyshenie-dolzhnostnyh-polnomochij/?page=42 37
Самоуправст

Разбой - судебная практика https://sudact.ru/practice/razboj/?page=47 28
Злоупотребление должностными полномочиями – судебная практика https://sudact.ru/practice/zloupotreblenie-dolzhnostnymi-polnomochiyami/?page=44 22
Превышение должностных полномочий - судебная практика https://sudact.ru/practice/prevyshenie-dolzhnostnyh-polnomochij/?page=49 37
Самоуправство - судебная практика https://sudact.ru/practice/samoupravstvo/?page=47 20
Присвоение и растрата - судебная практика https://sudact.ru/practice/prisvoenie-i-rastrata/?page=48 25
Приговор, неисполнение приговора - судебная практика https://sudact.ru/practice/prigovor-neispolnenie-prigovora/?page=49 50
Соучастие, предварительный сговор - судебная практика https://sudact.ru/practice/souchastie-predvaritelnyj-sgovor/?page=47 45
Умышленное причинение тяжкого вреда здоровью - судебная практика https://sudact.ru/practice/umyshlennoe-prichinenie-tyazhkogo-vreda-zdorovyu/?page=45 47
Уголовная ответственность несовершеннолетних - судебная пр

In [6]:
res = cur.execute("SELECT COUNT(*) FROM decisions;")
res.fetchall()

[(51304,)]

In [270]:
res = cur.execute("SELECT subsection_id, COUNT(*) FROM decisions GROUP BY subsection_id;")
res.fetchall()

[(1, 1000),
 (2, 1270),
 (3, 744),
 (4, 734),
 (5, 1290),
 (6, 1000),
 (7, 1280),
 (8, 1310),
 (9, 1000),
 (10, 1000),
 (11, 1000),
 (12, 1000),
 (13, 1040),
 (14, 22),
 (15, 20),
 (16, 90),
 (17, 22),
 (18, 12),
 (19, 1310),
 (20, 1000),
 (21, 1270),
 (22, 1000),
 (23, 1000),
 (24, 1000),
 (25, 1000),
 (26, 1300),
 (27, 1000),
 (28, 1000),
 (29, 1300),
 (30, 1000),
 (31, 1290),
 (32, 1300),
 (33, 1000),
 (34, 1000),
 (35, 1290),
 (36, 1000),
 (37, 1000),
 (38, 1000),
 (39, 1000),
 (40, 1270),
 (41, 1300),
 (42, 1000),
 (43, 1000),
 (44, 1000),
 (45, 1000),
 (46, 1000),
 (47, 1000),
 (48, 1000),
 (49, 1290),
 (50, 1000),
 (51, 1260),
 (52, 1290)]

In [271]:
res = cur.execute("SELECT * FROM decisions LIMIT 1;")
res.fetchall()

[(1,
  'Приговор № 1-13/2022 от 17 февраля 2022 г. по делу № 1-13/2022',
  '/regular/doc/chaeDyxsKiYN/',
  32,
  'Шумихинский районный суд (Курганская область) - Уголовное\n\n\n',
  '\n\n...Федерации" решение суда об изменении категории преступления с тяжкого на преступление средней тяжести позволяет суду при наличии оснований, предусмотренных статьями 75, 76, 78, 80.1,   84 ,  92, 94 УК РФ, освободить осужденного от отбывания назначенного наказания. Лицо, освобожденное от отбывания наказания по указанным основаниям, считается несудимым (часть 2 статьи 86 УК РФ).\nВ ...\n\n\n')]

In [23]:
class SaverDecisionContentPipeline:
    
    def process_item(self, item, spider):
       
        decision_id = item['decision_id']
        with open(f'decisions__data/{decision_id}.txt', 'w') as f:
            f.write(item['content'])
            
        return item

In [29]:
class SudactDecisionSpider(scrapy.Spider):
    """
    Парсинг содержимого решений в формате html 
    """
    def __init__(self):
        self.con = sqlite3.connect('sudactru.db')
        self.cur = self.con.cursor()
        
    name = "SudactDecisionSpider"
    allowed_domains = ['sudact.ru']
    
    res = cur.execute("SELECT href FROM decisions;")
    start_urls = list(map(lambda x: 'https://sudact.ru' + x[0], res.fetchall()))
        
    custom_settings = {
        'ITEM_PIPELINES': {
            '__main__.SaverDecisionContentPipeline': 1
        },
        "DOWNLOAD_DELAY": 1,
        "LOG_LEVEL": "WARNING",
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
    }

    def parse(self, response):
        current_url = response.request.url   
        h1 = response.xpath('//h1/text()').get()
        
        res = self.cur.execute("SELECT id FROM decisions WHERE href = ?", (
            current_url.replace('https://sudact.ru', ''),
        ))
        decision_id = res.fetchone()[0]
        
        html = response.xpath(r'//*[contains(@class, "h-content-wrap2")]/descendant-or-self::*[not(self::script or self::style)]/text()[normalize-space(.)]')
        text = '\n'.join(
            list(
                map(
                    lambda x: " ".join(
                                x.replace('\n', '').
                                replace('\t', '').
                                replace('\xa0', '').
                                split()
                            ), 
                    html.extract()
                )
            )[3:-10]
        )
        
        print(h1, current_url, decision_id, len(text))
        
        yield {'content': text,
               'decision_id': decision_id}

@wait_for(None)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(SudactDecisionSpider)
    return d

In [ ]:
run_spider()

Приговор № 1-13/2022 от 17 февраля 2022 г. по делу № 1-13/2022 https://sudact.ru/regular/doc/chaeDyxsKiYN/ 1 24143
Приговор № 1-141/2020 от 16 октября 2020 г. по делу № 1-102/2020 https://sudact.ru/regular/doc/h0zs5sMOlsJv/ 6 31967
Приговор № 1-227/2020 от 25 сентября 2020 г. https://sudact.ru/regular/doc/Z3pBtpg0WrlJ/ 7 19514
Апелляционное постановление № 22-2294/2020 от 15 сентября 2020 г. по делу № 4/17-339/2020 https://sudact.ru/regular/doc/kU0i9wcvEjQt/ 8 4697
Апелляционное постановление № 22-4186/2020 от 25 августа 2020 г. по делу № 1-157/2020 https://sudact.ru/regular/doc/hZjRLKrnmDVQ/ 9 9703
Апелляционное постановление № 22-2472/2020 от 10 августа 2020 г. по делу № 1-266/20 https://sudact.ru/regular/doc/U1nQj4k0YUWZ/ 10 22521
Приговор № 1-3/2022 1-88/2021 от 16 февраля 2022 г. по делу № 1-3/2022 https://sudact.ru/regular/doc/rytByFnoNErl/ 18 79308
Приговор № 1-28/2022 от 16 февраля 2022 г. по делу № 1-28/2022 https://sudact.ru/regular/doc/hL2cLZs8Y9Zx/ 19 26143
Приговор № 1-200

Решение № 2А-162/2021 2А-162/2021~М-174/2021 М-174/2021 от 25 июня 2021 г. по делу № 2А-162/2021 https://sudact.ru/regular/doc/hpa7hgaYr2mt/ 160 11330
Апелляционное постановление № 22-2124/2022 от 29 сентября 2022 г. по делу № 4/12-60/2022 https://sudact.ru/regular/doc/IF3uxBWGHadu/ 161 5234
Апелляционное постановление № 22-1920/2022 от 6 сентября 2022 г. по делу № 22-1920/2022 https://sudact.ru/regular/doc/TgBx1wy16s1n/ 162 14471
Приговор № 1-407/2021 от 26 ноября 2021 г. по делу № 1-407/2021 https://sudact.ru/regular/doc/64IKmtEaW6JC/ 170 9865
Апелляционное постановление № 22-1797/2020 от 10 августа 2020 г. по делу № 1-10/2020 https://sudact.ru/regular/doc/AP7UEskrfgNh/ 171 10919
Апелляционное постановление № 22-1765/2020 от 30 июля 2020 г. по делу № 4/4-2/2020 https://sudact.ru/regular/doc/jZHVzAwKLHNt/ 172 8898
Апелляционное постановление № 22-1132/2020 от 30 июля 2020 г. по делу № 4/4-1/2020 https://sudact.ru/regular/doc/m6UDHWZAjoQT/ 173 5175
Приговор № 1-1/2022 1-12/2021 1-213/2

Решение № 12-298/2019 от 18 июня 2019 г. по делу № 12-298/2019 https://sudact.ru/regular/doc/FxPeNUqWMprv/ 259 21125
Решение № 12-304/2019 от 23 мая 2019 г. по делу № 12-304/2019 https://sudact.ru/regular/doc/JgDgpdogjRYx/ 260 16147
Апелляционное постановление № 22-1775/2022 от 23 августа 2022 г. по делу № 1-106/2022 https://sudact.ru/regular/doc/jItpQ6T7IgVm/ 261 5647
Апелляционное постановление № 22-1561/2022 от 27 июля 2022 г. по делу № 4/17-53/2022 https://sudact.ru/regular/doc/1jsHly5dhLNo/ 262 6277
Апелляционное постановление № 22-1377/2022 от 27 июня 2022 г. по делу № 4/15-53/2022 https://sudact.ru/regular/doc/nP8iZ3KXePbK/ 263 7532
Апелляционное постановление № 22-1251/2022 от 15 июня 2022 г. по делу № 1-138/2022 https://sudact.ru/regular/doc/b05BppyRcCx1/ 264 7212
Апелляционное постановление № 22-1182/2022 от 6 июня 2022 г. по делу № 1-125/2022 https://sudact.ru/regular/doc/0auHckztoPdd/ 265 8012
Апелляционное постановление № 22-1113/2022 от 3 июня 2022 г. по делу № 4/15-31/20

Решение № 2А-1042/2021 2А-1042/2021~М-305/2021 М-305/2021 от 18 марта 2021 г. по делу № 2А-1042/2021 https://sudact.ru/regular/doc/5jev5Id6jnQO/ 323 7242
Приговор № 1-153/2020 1-7/2021 от 18 марта 2021 г. по делу № 1-153/2020 https://sudact.ru/regular/doc/iZocXJmD21od/ 324 70932
Постановление № 1-17/2021 от 18 марта 2021 г. по делу № 1-17/2021 https://sudact.ru/regular/doc/34YG0jtpwkmC/ 325 20134
Апелляционное постановление № 22-2346/2020 от 29 декабря 2020 г. по делу № 4/1-14/2020 https://sudact.ru/regular/doc/EPMB2IaI31R/ 326 8653
Апелляционное постановление № 22-1546/2020 от 28 декабря 2020 г. по делу № 1-117/2020 https://sudact.ru/regular/doc/Y9qflxgexc15/ 327 19074
Апелляционное постановление № 22-2457/2020 22К-2457/2020 от 24 декабря 2020 г. по делу № 1-64/2020 https://sudact.ru/regular/doc/zeSH3KFGPBWN/ 328 10921
Апелляционное постановление № 22-2397/2020 от 23 декабря 2020 г. по делу № 4/17-553/2020 https://sudact.ru/regular/doc/kk6A55joevoQ/ 329 7655
Апелляционное постановлени

Апелляционное постановление № 22-2466/2020 22К-2466/2020 от 24 декабря 2020 г. по делу № 1-59/2020 https://sudact.ru/regular/doc/KHpjAUWV3bJ4/ 414 9500
Апелляционное постановление № 22-2469/2020 22К-2469/2020 от 24 декабря 2020 г. по делу № 3/2-54/2020 https://sudact.ru/regular/doc/L2hosc9X1yUV/ 415 10561
Апелляционное постановление № 22К-1740/2020 УК-22-1740/2020 от 23 декабря 2020 г. по делу № 3/2-69/2020 https://sudact.ru/regular/doc/AA1kSmlYxdmm/ 416 5622
Апелляционное постановление № 22К-1741/2020 УК-22-1741/2020 от 23 декабря 2020 г. по делу № 3/2-1-26/2020 https://sudact.ru/regular/doc/qefiLDM790rT/ 417 6397
Апелляционное постановление № 22-1716/2020 22К-1716/2020 УК22-1716/2020 от 21 декабря 2020 г. по делу № 3/2-995/2020 https://sudact.ru/regular/doc/RZukBVWEhuJt/ 418 10191
Апелляционное постановление № 22К-1717/2020 УК-22-1717/2020 от 21 декабря 2020 г. по делу № 3/2-1005/2020 https://sudact.ru/regular/doc/cfphty8Sp3W4/ 419 9252
Апелляционное постановление № 22К-3363/2020 от 

...

Приговор № 1-332/2020 от 30 июля 2020 г. по делу № 1-332/2020 https://sudact.ru/regular/doc/wgZunhC1T65X/ 4338 11817
Решение № 2А-1298/2020 2А-1298/2020~М-1141/2020 М-1141/2020 от 30 июля 2020 г. по делу № 2А-1298/2020 https://sudact.ru/regular/doc/YTRtsOajDa1j/ 4339 7490
Решение № 2А-545/2020 2А-545/2020~М-482/2020 М-482/2020 от 30 июля 2020 г. по делу № 2А-545/2020 https://sudact.ru/regular/doc/Rqsx9jBKDt6l/ 4340 9154
Решение № М-26/2014 2-229/2014 2-229/2014~М-26/2014 от 31 января 2014 г. https://sudact.ru/regular/doc/XIj4E0GlGaVC/ 4341 9030
Решение № М-31/2014 2-204/14 2-204/2014~М-31/2014 2-204/2014 от 23 января 2014 г. https://sudact.ru/regular/doc/HOLXbdkoFGbQ/ 4342 7853
Апелляционное определение № 33-859/2014 от 23 января 2014 г. по делу № 33-859/2014 https://sudact.ru/regular/doc/nWKZf19AuX6m/ 4343 14186
Постановление № 4А-743/2013 от 20 января 2014 г. по делу № 4А-743/2013 https://sudact.ru/regular/doc/OwGGfQ3faZrp/ 4344 13184
Решение № 12-10/2014 12-423/2013 от 10 января 201

Решение № 2А-3454/2020 2А-3454/2020~М-2636/2020 М-2636/2020 от 14 июля 2020 г. по делу № 2А-3454/2020 https://sudact.ru/regular/doc/7SWjppXC7D6R/ 4403 7330
Решение № 12-472/2020 от 14 июля 2020 г. по делу № 12-472/2020 https://sudact.ru/regular/doc/yUIINBW50poJ/ 4404 12744
Решение № 12-254/2020 от 13 июля 2020 г. по делу № 12-254/2020 https://sudact.ru/regular/doc/5aaNOfmjm2gJ/ 4405 10484
Решение № 2А-2629/2020 2А-2629/2020~М-1875/2020 М-1875/2020 от 13 июля 2020 г. по делу № 2А-2629/2020 https://sudact.ru/regular/doc/xpTdYQaizwon/ 3612 14321
Постановление № 5-115/2020 от 13 июля 2020 г. по делу № 5-115/2020 https://sudact.ru/regular/doc/6R1rTKK9DZb/ 4407 5804
Постановление № 5-113/2020 от 13 июля 2020 г. по делу № 5-113/2020 https://sudact.ru/regular/doc/AcofU5eIJwlY/ 4408 5975
Решение № 2А-2625/2020 2А-2625/2020~М-1877/2020 М-1877/2020 от 13 июля 2020 г. по делу № 2А-2625/2020 https://sudact.ru/regular/doc/CMZT5P3bjKbw/ 3615 14320
Решение № 12-60/2020 от 13 июля 2020 г. по делу № 12-

Апелляционное постановление № 22-708/2020 от 10 июня 2020 г. https://sudact.ru/regular/doc/lXQJ1SnZ9Bl5/ 4488 14130
Апелляционное постановление № 22К-209/2020 от 10 июня 2020 г. по делу № №3/1-24/2020 https://sudact.ru/regular/doc/iARQhn5bGeZQ/ 4489 17478
Апелляционное постановление № 22К-689/2020 от 9 июня 2020 г. по делу № 3/10-107/2020 https://sudact.ru/regular/doc/OMbbOyp7gJoV/ 4490 14998
Решение № 2А-371/2020 2А-371/2020~М-327/2020 М-327/2020 от 24 июля 2020 г. по делу № 2А-371/2020 https://sudact.ru/regular/doc/ksf2pXAI5u6b/ 4491 14328
Решение № 2А-370/2020 2А-370/2020~М-328/2020 М-328/2020 от 24 июля 2020 г. по делу № 2А-370/2020 https://sudact.ru/regular/doc/83C2LO2ikvN5/ 4492 14271
Решение № 2А-363/2020 2А-363/2020~М-335/2020 М-335/2020 от 24 июля 2020 г. по делу № 2А-363/2020 https://sudact.ru/regular/doc/BapLM94TIkJ/ 4493 14264
Решение № 2А-360/2020 2А-360/2020~М-338/2020 М-338/2020 от 24 июля 2020 г. по делу № 2А-360/2020 https://sudact.ru/regular/doc/sqVj3cKPqxoM/ 4494 142

Решение № 2А-287/2017 2А-287/2017~М-87/2017 М-87/2017 от 21 июня 2017 г. по делу № 2А-287/2017 https://sudact.ru/regular/doc/zpqGx3pWwTVV/ 4550 27826
Приговор № 1-513/2019 от 3 сентября 2019 г. по делу № 1-513/2019 https://sudact.ru/regular/doc/KA783DHaPGY0/ 4551 10218
Приговор № 1-1/2019 1-10/2014 1-2/2017 1-2/2018 1-3/2016 1-324/2013 1-4/2015 от 3 сентября 2019 г. по делу № 1-1/2019 https://sudact.ru/regular/doc/cZwaN4Skht1S/ 4552 8005
Постановление № 1-162/2019 от 2 сентября 2019 г. по делу № 1-162/2019 https://sudact.ru/regular/doc/gTOAzkqyhhVT/ 4553 29265
Апелляционное постановление № 22-1481/2019 от 29 августа 2019 г. по делу № 22-1481/2019 https://sudact.ru/regular/doc/5QAUiBszGnK/ 4554 9339
Постановление № 1-336/2019 от 29 августа 2019 г. по делу № 1-336/2019 https://sudact.ru/regular/doc/LrCrqUTT9nxd/ 4555 13225
Постановление № 44У-115/2019 4У-1313/2018 4У-17/2019 от 28 августа 2019 г. по делу № 1-170/2015 https://sudact.ru/regular/doc/T0LGjvSSHwJ/ 4556 12074
Приговор № 1-372/

Апелляционное постановление № 22-882/2021 от 19 мая 2021 г. https://sudact.ru/regular/doc/jsTwjmfwWd5U/ 4612 12670
Апелляционное постановление № 22-868/2021 от 13 мая 2021 г. https://sudact.ru/regular/doc/qj12QvA0xzlQ/ 4613 6011
Апелляционное постановление № 22-823/2021 от 11 мая 2021 г. https://sudact.ru/regular/doc/VIMuffkGu2H4/ 4614 9132
Апелляционное постановление № 22-812/2021 от 6 мая 2021 г. https://sudact.ru/regular/doc/cKwqgaucZa1Q/ 4615 25319
Апелляционное постановление № 22-752/2021 от 28 апреля 2021 г. по делу № 1-31/2021 https://sudact.ru/regular/doc/qKB8GJCANAVp/ 4616 6853
Апелляционное постановление № 22-722/2021 от 26 апреля 2021 г. по делу № 4/16-32/2021 https://sudact.ru/regular/doc/HtGlbZuYlyVK/ 4617 6799
Апелляционное постановление № 22-673/2021 от 14 апреля 2021 г. по делу № 1-79/2021 https://sudact.ru/regular/doc/dQJI0ODgAqHd/ 4618 10038
Апелляционное постановление № 22-625/2021 от 14 апреля 2021 г. по делу № 1-76/2021 https://sudact.ru/regular/doc/lliNPe2wchO/ 46

In [28]:
# res = cur.execute("SELECT * FROM decisions__data LIMIT 1;")
# res.fetchall()

In [441]:
# list(map(lambda x: x.replace('\n', '').replace('\t', '').replace('\xa0', ''), html.extract()))[3:-10]

['    Дело № 1- 13/2022',
 'Уникальный идентификатор дела: 45RS0023-01-2022-000129-83',
 'ПРИГОВОР',
 'Именем Российской Федерации',
 'город Шумиха 17 февраля 2022 года',
 'Шумихинский районный суд Курганской области в составе председательствующего судьи Пономарева Д.В.,',
 'с участием государственного обвинителя Антонова А.В.,',
 'подсудимой ФИО1,',
 'защитника подсудимой адвоката Недомец Е.В.,',
 'при секретаре Павловой А.В.,',
 'рассмотрев в открытом судебном заседании в г. Шумихе Шумихинского района Курганской области уголовное дело в отношении',
 'ФИО1, несудимой,',
 'обвиняемой в совершении преступления, предусмотренного п. «г» ч. 3 ст. ',
 '158 УК РФ',
 ',',
 'УСТАНОВИЛ:',
 'ФИО1 совершила кражу, то есть тайное хищение чужого имущества, совершенную с банковского счета, при следующих обстоятельствах.',
 'В период c 03 сентября по 05 сентября 2021 года ФИО1 в г. Шумихе Шумихинского района Курганской области, воспользовавшись тем, что в ее фактическом владении находится банковская 

In [445]:
# print('\n'.join(
#     list(
#         map(
#             lambda x: " ".join(
#                         x.replace('\n', '').
#                         replace('\t', '').
#                         replace('\xa0', '').
#                         split()
#                     ), 
#             html.extract()
#         )
#     )[3:-10]
# ))

Дело № 1- 13/2022
Уникальный идентификатор дела: 45RS0023-01-2022-000129-83
ПРИГОВОР
Именем Российской Федерации
город Шумиха 17 февраля 2022 года
Шумихинский районный суд Курганской области в составе председательствующего судьи Пономарева Д.В.,
с участием государственного обвинителя Антонова А.В.,
подсудимой ФИО1,
защитника подсудимой адвоката Недомец Е.В.,
при секретаре Павловой А.В.,
рассмотрев в открытом судебном заседании в г. Шумихе Шумихинского района Курганской области уголовное дело в отношении
ФИО1, несудимой,
обвиняемой в совершении преступления, предусмотренного п. «г» ч. 3 ст.
158 УК РФ
,
УСТАНОВИЛ:
ФИО1 совершила кражу, то есть тайное хищение чужого имущества, совершенную с банковского счета, при следующих обстоятельствах.
В период c 03 сентября по 05 сентября 2021 года ФИО1 в г. Шумихе Шумихинского района Курганской области, воспользовавшись тем, что в ее фактическом владении находится банковская карта АО «Почта Банк» на имя Л., ранее утерянная О. и самостоятельной матер

In [417]:
# class DecisionClearing(object):
#     def process_item(self, item, spider):
#         # удаляем рекламу
#         # удаляем скрипты
#         # html переносы заменяем на обычные (текстовые)
#         # удаляем остальные html теги        
        
#         print(item)
        
#         item['value'] = self.__remove_html_tags__(item['value'])
#         return item
#         # .css('::text').getall()
# #         lines = dict(item)["quote"].splitlines()
# #         first_line = self.__remove_html_tags__(lines[0])
# #         return {'quote': first_line}

#     def __remove_html_tags__(self, text):
#         """remove html tags from string"""
#         html_tags = re.compile('<.*?>')
#         return re.sub(html_tags, '', text)

In [184]:
# res = cur.execute("SELECT href FROM subsections;")
# res.fetchall()

In [185]:
# current_url = 'https://sudact.ru/practice/amnistiya/'

In [186]:
# current_url.split('?')[0]

In [187]:
# res = cur.execute("SELECT id FROM subsections WHERE href = ?", (current_url.replace('https://sudact.ru', '').split('?')[0],))
# subsection_id = res.fetchone()
# print(subsection_id)

In [188]:
# 'https://sudact.ru/practice/po-zashite-prav-potrebitelej/?page=5'.split('?')[0]

In [114]:
#cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
#print(cur.fetchall())

In [82]:
# res = cur.execute("SELECT id FROM sections WHERE href = ?", ('https://sudact.ru/practice/sudebnaya-praktika-po-ugolovnym-delam/',))
# section_id = res.fetchone()
# section_id

In [ ]:
            
# class ExtractFirstLine(object):
#     def process_item(self, item, spider):
#         """text processing"""
#         lines = dict(item)["quote"].splitlines()
#         #print(lines)
#         first_line = self.__remove_html_tags__(lines[0])

#         return {'quote': first_line}

#     def __remove_html_tags__(self, text):
#         """remove html tags from string"""
#         html_tags = re.compile('<.*?>')
#         return re.sub(html_tags, '', text)